In [14]:
import pandas as pd
import numpy as np

In [15]:
df = pd.DataFrame(columns=['text','twss'])

In [16]:
df

,text,twss


In [17]:
with open('data/twss-stories-parsed.txt') as f:
    for sentence in f.readlines():
        df = df.append({
            'text': sentence.strip(),
            'twss': 1
        }, ignore_index=True)

In [18]:
df

,text,twss
0,you have to unhinge your jaw to get it all in,1
1,i want to go down i want to go down,1
2,you need to get wet,1
3,it's twice as long as it is wide,1
4,i've never been this wet before,1
...,...,...
2086,just close your eyes and shove it in your mout...,1
2087,it was coming at my face,1
2088,will you be wet forever,1
2089,i like the long thick black ones,1


In [19]:
with open('data/fmylife-parsed.txt') as f:
    for sentence in f.readlines():
        df = df.append({
            'text': sentence.strip(),
            'twss': 0
        }, ignore_index=True)

In [20]:
df

,text,twss
0,you have to unhinge your jaw to get it all in,1
1,i want to go down i want to go down,1
2,you need to get wet,1
3,it's twice as long as it is wide,1
4,i've never been this wet before,1
...,...,...
5437,why am i annoyed at him because he wants to go...,0
5438,the only person who wished me a happy birthday...,0
5439,my doctor told me my asthma was being triggere...,0
5440,he wrote a note to my dad asking him to refrai...,0


In [21]:
df['twss'].value_counts()

0    3351
1    2091
Name: twss, dtype: int64

In [22]:
df.describe()

,text,twss
count,5442,5442
unique,5427,2
top,its my birthday,0
freq,3,3351


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

In [24]:
X=df['text']
y=df['twss']

In [25]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

In [36]:
X_train_counts.shape

(3809, 4990)

In [44]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [54]:
y_train=np.array(y_train)

In [56]:
y_train

array([0, 1, 1, ..., 1, 0, 0], dtype=object)

In [57]:
y_train=y_train.astype('int')

In [58]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [121]:
docs_new = ['It\'s hard.', 'they will be held against you']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

In [122]:
predicted

array([1, 1])

In [85]:
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

y_predicted=clf.predict(X_test_tfidf)

In [97]:
X_test

5262    while trying on an outfit in the dressing room...
5411    i once again told my son he needed a job and a...
3937                                          im nineteen
1442                            i need another six inches
3561                      i got stuck in a small elevator
                              ...                        
3897      i said nothing because he hates being corrected
3083    the wind snapped it wide open then immediately...
23      pull out the stick wipe it off then put it bac...
1                     i want to go down i want to go down
4869    the highlight of my day was learning how to dr...
Name: text, Length: 1633, dtype: object

In [100]:
for i, result in enumerate(y_predicted):
    if result==1:
        print(X_test.iloc[i], y_test.iloc[i])

i need another six inches 1
i got stuck in a small elevator 0
come on put your fist in 1
emily pull out twss she then replied i know i know i never do its a habit 1
you just gotta stick it in there and move it around until the hole gets bigger and that makes it easier to put it in next time 1
its not how hard you blow its all about technique 1
i love the taste but its just too big to fit in my mouth 1
yea ive been blowing all day and it just keeps on coming 1
it is so hard and impossible to make it very long my grandma said ever so supportingly dont think of it as hard think of it as a fun experience 1
but i dont wanna bend over im all sticky 1
oh my god its way to big it wont fit 1
after which she says oh not again 0
leaving you unable to open your eyes 0
you may be able to get it to fit but it is a really tight space 1
why the f*ck would you pull out right when im coming 1
ahh it hit me right in the face 1
i felt it penetrate but once it was in and it released all the liquidy stuff i

In [86]:
np.mean(y_predicted == y_test)

0.9571341090018372

In [87]:
import joblib

In [88]:
joblib.dump(clf, 'twss_model.sav')

['twss_model.sav']

In [89]:
clf_loaded = joblib.load('twss_model.sav')

In [90]:
docs_new = ['It\'s hard.', 'they will be held against you']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf_loaded.predict(X_new_tfidf)